## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [ ]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
message = "Latest AI Agent frameworks in 2025. Responda em portugues. Seja suscinto."

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [3]:

##Criar um Novo Agente Clarificador

class ClarificationQuestions(BaseModel):
    questions: list[str] = Field(description="3 perguntas para esclarecer melhor a consulta")

CLARIFICATION_INSTRUCTIONS = """Você é um assistente que ajuda a refinar consultas de pesquisa. 
Dada uma consulta inicial, gere exatamente 3 perguntas esclarecedoras que ajudariam a 
tornar a pesquisa mais específica e útil. As perguntas devem cobrir aspectos como: 
escopo temporal, público-alvo, contexto específico, ou detalhes técnicos."""

clarification_agent = Agent(
    name="ClarificationAgent",
    instructions=CLARIFICATION_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ClarificationQuestions,
)

#########

# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for. Responda em portugues."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [4]:

message = "Latest AI Agent frameworks in 2025. Responda em portugues."

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='Para encontrar informações atualizadas sobre os frameworks de agentes de IA que estão sendo desenvolvidos ou utilizados em 2025.', query='frameworks agentes de IA 2025'), WebSearchItem(reason='Para identificar quais tecnologias e abordagens estão sendo mais adotadas na criação de agentes inteligentes em 2025.', query='tendências agentes de IA 2025'), WebSearchItem(reason='Para obter um panorama geral das principais ferramentas e plataformas disponíveis para desenvolvimento de agentes de IA em 2025.', query='principais ferramentas desenvolvimento IA 2025')]


In [5]:
print(result)

RunResult:
- Last agent: Agent(name="PlannerAgent", ...)
- Final output (WebSearchPlan):
    {
      "searches": [
        {
          "reason": "Para encontrar informações atualizadas sobre os frameworks de agentes de IA que estão sendo desenvolvidos ou utilizados em 2025.",
          "query": "frameworks agentes de IA 2025"
        },
        {
          "reason": "Para identificar quais tecnologias e abordagens estão sendo mais adotadas na criação de agentes inteligentes em 2025.",
          "query": "tendências agentes de IA 2025"
        },
        {
          "reason": "Para obter um panorama geral das principais ferramentas e plataformas disponíveis para desenvolvimento de agentes de IA em 2025.",
          "query": "principais ferramentas desenvolvimento IA 2025"
        }
      ]
    }
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [6]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("williamlapa40@gmail.com") # Change this to your verified email
    to_email = To("williamlapa@hotmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [7]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001835BB87CE0>, strict_json_schema=True)

In [8]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [9]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words. Escreva em portugues. Traga referências em forma de links ao final do texto"
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [10]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [ ]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

# ADICIONE AQUI AS NOVAS FUNÇÕES:
async def ask_clarifications(query: str):
    """Gera 3 perguntas esclarecedoras baseadas na consulta"""
    print("Gerando perguntas esclarecedoras...")
    result = await Runner.run(clarification_agent, f"Consulta: {query}")
    return result.final_output

async def refine_search_plan(original_query: str, questions: list[str], answers: list[str]):
    """Refina o plano de busca com base nas respostas das perguntas"""
    context = f"""
    Consulta original: {original_query}
    
    Informações adicionais baseadas nas perguntas esclarecedoras:
    {chr(10).join([f"- {q}: {a}" for q, a in zip(questions, answers)])}
    
    Use essas informações para criar um plano de busca mais refinado e específico.
    """
    result = await Runner.run(planner_agent, context)
    return result.final_output

### Showtime!

In [ ]:
query = "Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    
    # NOVO: Gerar perguntas esclarecedoras
    clarifications = await ask_clarifications(query)
    print("\nPerguntas esclarecedoras:")
    for i, question in enumerate(clarifications.questions, 1):
        print(f"{i}. {question}")
    
    # Para teste, vamos simular respostas
    # Em produção, você coletaria estas respostas do usuário
    sample_answers = [
        "Foco em frameworks open-source para Python",
        "Para desenvolvedores iniciantes e intermediários", 
        "Incluir comparação de facilidade de uso e documentação"
    ]
    
    print("\nRespostas simuladas:")
    for i, answer in enumerate(sample_answers, 1):
        print(f"{i}. {answer}")
    
    # Refinar o plano de busca com as informações coletadas
    refined_search_plan = await refine_search_plan(query, clarifications.questions, sample_answers)
    
    # Continuar com o fluxo normal usando o plano refinado
    search_results = await perform_searches(refined_search_plan)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Hooray!")




### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>

## Desafios

Start by coming up with 3 clarifyng questions based on the query. (Comece elaborando 3 perguntas de esclarecimento com base na solicitação.)

Tune the searches taking into account the clarifications. (Ajuste as buscas levando em consideração os esclarecimentos.)

Make the manager an Agent with agent_as-tolls and handoffs.(Transforme o gerente em um Agente com agent_as-tools e handsoff).


## O que o código passará a fazer:

**Antes das mudanças:**
- Recebia uma consulta → planejava buscas → executava buscas → escrevia relatório → enviava email

**Depois das mudanças:**
1. **Recebe a consulta inicial** ("Latest AI Agent frameworks in 2025")

2. **Gera 3 perguntas esclarecedoras automaticamente**, como:
   - "Você quer focar em frameworks específicos para alguma linguagem?"
   - "Qual é o público-alvo do relatório?"
   - "Há algum aspecto específico que deve ser priorizizado?"

3. **Usa respostas simuladas** (ou reais do usuário) para refinar a busca

4. **Cria um plano de busca mais específico** baseado nas respostas, resultando em buscas mais direcionadas

5. **Continua o fluxo normal** (busca → relatório → email) mas com resultados mais precisos

## Resultado Prático:

**Ao invés de buscas genéricas como:**
- "AI Agent frameworks 2025"
- "Latest AI tools"
- "Agent development platforms"

**Terá buscas refinadas como:**
- "Open-source Python AI agent frameworks 2025"
- "Beginner-friendly AI agent development tools"
- "AI agent frameworks comparison usability documentation"

**Benefício:** O relatório final será mais específico, relevante e útil, porque as buscas foram direcionadas pelas perguntas esclarecedoras, ao invés de serem genéricas.

Claro! Aqui está um resumo conceitual do fluxo do código do notebook **5_lab4_desafio.ipynb**:

---

## Passo a Passo Conceitual

### 1. **Receber a Consulta Inicial**
O usuário fornece uma consulta de pesquisa, por exemplo:  
`"Latest AI Agent frameworks in 2025"`

---

### 2. **Gerar Perguntas Esclarecedoras**
Um agente (ClarificationAgent) gera automaticamente 3 perguntas para refinar a consulta, tornando a busca mais específica e relevante.

---

### 3. **Obter Respostas para as Perguntas**
As respostas às perguntas esclarecedoras são coletadas (no exemplo, são simuladas, mas poderiam ser do usuário).

---

### 4. **Refinar o Plano de Busca**
Com base nas respostas, um novo plano de busca é criado por outro agente (PlannerAgent), tornando as buscas mais direcionadas e eficazes.

---

### 5. **Executar as Buscas**
Para cada item do plano de busca refinado, um agente de busca (SearchAgent) realiza pesquisas na web e retorna os resultados.

---

### 6. **Gerar Relatório**
Um agente escritor (WriterAgent) recebe os resultados das buscas e gera um relatório detalhado e estruturado em Markdown, incluindo um resumo, o conteúdo principal e sugestões de pesquisas futuras.

---

### 7. **Enviar o Relatório por Email**
Um agente de email (EmailAgent) envia o relatório gerado para o destinatário especificado, formatando-o em HTML.

---

## Resumindo em Fluxo:

1. **Consulta inicial**  
2. → **Perguntas esclarecedoras**  
3. → **Respostas**  
4. → **Plano de busca refinado**  
5. → **Execução das buscas**  
6. → **Geração do relatório**  
7. → **Envio do relatório por email**

---

**Benefício:**  
O processo resulta em pesquisas e relatórios muito mais relevantes e personalizados, pois as buscas são ajustadas conforme as necessidades e contexto do usuário, graças ao ciclo de esclarecimento e refinamento.